In [ ]:
function ConnectButton() {
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton, 60000);

In [1]:
path_root = '/content/drive/My Drive/Colab Notebooks/DU/data/'
path_train = path_root + 'Corona_NLP_train.csv'
path_test = path_root + 'Corona_NLP_test.csv'
path_models = path_root + 'models/'
path_logs = path_root + 'logs/'

### Step 1: Data preparation

In [2]:
!pip install unidecode

     |████████████████████████████████| 245kB 17.5MB/s 


In [3]:
import nltk
import html
import unidecode
from string import ascii_lowercase
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')

def clean_text(df):
    df['x'] = [html.unescape(x) for x in df['x']]
    df['x'] = [re.sub(r'https?://\S+', '', x) for x in df['x']]
    df['x'] = [re.sub(r'[^\w\s]|\d+', '', x) for x in df['x']]
    df['x'] = [re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', x) for x in df['x']]
    df['x'] = [re.sub(r'\s\s+|_|\'', ' ', x) for x in df['x']]
    df['x'] = [x.strip().lower() for x in df['x']]
    df['x'] = [unidecode.unidecode(x) for x in df['x']]

    for c in ascii_lowercase:
        df['x'] = [re.sub(c+'{3,}', c+c, x) for x in df['x']]

    #df['x'] = [regexp_tokenize(x, '\w+') for x in df['x']]
    #df['x'] = [' '.join(w for w in x if not w in stopwords.words('english')) for x in df['x']]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
import re
import pandas as pd

def load_csv(path):
    df = pd.read_csv(path, encoding='latin')
    df = df.drop(columns=['UserName', 'ScreenName', 'Location', 'TweetAt'])
    df = df.rename(columns={'OriginalTweet':'x', 'Sentiment':'y'})

    df['y'] = df['y'].apply(lambda x: re.sub('Extremely ', '', x))

    clean_text(df)

    return df

### Step 2: Data exploration

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

y_count = Counter(df['y'])
plt.figure(figsize=(20, 5))
plt.pie(y_count.values(), labels=[class_trans[x] for x in y_count.keys()], autopct='%1.1f%%')
plt.show()

NameError: ignored

<Figure size 1440x360 with 0 Axes>

In [ ]:
from wordcloud import WordCloud

for c in classes:
    x = df[df['y'] == class_trans[c]]['x'].to_string()
    plt.imshow(WordCloud().generate(x))
    plt.show()

### Classifier 1: Logistic regression

In [ ]:
%%time

train, test = load_csv(path_train), load_csv(path_test)

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

model = Pipeline([('vectorizer', TfidfVectorizer()),
                  ('clf', LogisticRegression(max_iter=500))])
model.fit(train['x'], train['y'])

from sklearn.metrics import classification_report

y_pred = model.predict(test['x'])
print(classification_report(test['y'], y_pred))

CPU times: user 3min 53s, sys: 5min 13s, total: 9min 6s
Wall time: 4min 44s


### Classifier 2: DistilBERT

In [5]:
!pip install transformers

     |████████████████████████████████| 1.5MB 7.5MB/s 
     |████████████████████████████████| 2.9MB 16.0MB/s 
     |████████████████████████████████| 890kB 40.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=bcd574e3a3f16cb5c3137efa11db93f6efacc33e153ca304e74edeba85773ac7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [8]:
MODEL_NAME = 'DistilBERT'

!rm -rf '{path_models}{MODEL_NAME}' '{path_logs}{MODEL_NAME}'

from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train, test = load_csv(path_train), load_csv(path_test)

data_train = tokenizer.batch_encode_plus(
    train['x'].tolist(),
    truncation=True,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=50,
    return_tensors='pt'
)

data_test = tokenizer.batch_encode_plus(
    test['x'].tolist(),
    truncation=True,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=50,
    return_tensors='pt'
)

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

train['y_encd'] = encoder.fit_transform(train['y'])
test['y_encd'] = encoder.fit_transform(test['y'])

import tensorflow as tf

dataset_train = tf.data.Dataset.from_tensor_slices((
    dict(data_train),
    train['y_encd'].tolist()
))

dataset_test = tf.data.Dataset.from_tensor_slices((
    dict(data_test),
    test['y_encd'].tolist()
))

from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

args = TFTrainingArguments(
    output_dir=path_models+MODEL_NAME,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=path_logs+MODEL_NAME,
    logging_steps=10
)

with args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

trainer = TFTrainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7ffb312bde58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7ffb312bde58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [ ]:
%%time

trainer.train()
trainer.save_model()

### Classifier 3: DistilGPT2

In [5]:
!pip install transformers

     |████████████████████████████████| 1.5MB 19.2MB/s 
     |████████████████████████████████| 890kB 22.0MB/s 
     |████████████████████████████████| 2.9MB 60.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=590dc7ab9e9424df2ccdd71cd7ac865013cff134a81b0c209d630c59f68ce47b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [9]:
MODEL_NAME = 'DistilGPT2'

!rm -rf '{path_models}{MODEL_NAME}' '{path_logs}{MODEL_NAME}'

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

train, test = load_csv(path_train), load_csv(path_test)

data_train = tokenizer.batch_encode_plus(
    train['x'].tolist(),
    truncation=True,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=50,
    return_tensors='pt'
)

data_test = tokenizer.batch_encode_plus(
    test['x'].tolist(),
    truncation=True,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=50,
    return_tensors='pt'
)

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

train['y_encd'] = encoder.fit_transform(train['y'])
test['y_encd'] = encoder.fit_transform(test['y'])

import tensorflow as tf

dataset_train = tf.data.Dataset.from_tensor_slices((
    dict(data_train),
    train['y_encd'].tolist()
))

dataset_test = tf.data.Dataset.from_tensor_slices((
    dict(data_test),
    test['y_encd'].tolist()
))

from transformers import TFAutoModelForSequenceClassification, TFTrainer, TFTrainingArguments

args = TFTrainingArguments(
    output_dir=path_models+MODEL_NAME,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=path_logs+MODEL_NAME,
    logging_steps=10
)

with args.strategy.scope():
    model = tokenizer = TFAutoModelForSequenceClassification.from_pretrained('distilgpt2', num_labels=3)

trainer = TFTrainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


All model checkpoint layers were used when initializing TFGPT2ForSequenceClassification.

Some layers of TFGPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
%%time

trainer.train()
trainer.save_model()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f3245e5be58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f3245e5be58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
